# AntiPhish-LLM

This is a demo of AntiPhish-LLM, a tool that leverages OpenAI's GPT models for the automatic classification of phishing emails and generation of warning messages.

# Setup

Firstly, we install the needed python libraries.

In [ ]:
!pip install scipy
!pip install tenacity
!pip install cohere
!pip install tiktoken
!pip install openai
!pip install requests
!pip install evals
!pip install beautifulsoup4
!pip install dnspython

Then, be sure to fill the **.env** file with the keys for the third-party services.

# Email Preprocessing

Let's import the Preprocessor module for the preprocessing of the emails.

We pre-process the email following the approach used in [K. Misra and J. T. Rayz, "LMs go Phishing: Adapting Pre-trained Language Models to Detect Phishing Emails," 2022 IEEE/WIC/ACM International Joint Conference on Web Intelligence and Intelligent Agent Technology (WI-IAT), Niagara Falls, ON, Canada, 2022, pp. 135-142, doi: 10.1109/WI-IAT55865.2022.00028.](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10101955)

In [ ]:
import preprocessor

# URL Enricher

We define the functions to collect online information about the URL(s) in the email.

In [ ]:
import url_enricher

Let's finally open an email and preprocess it.
For now, we gather only URL info for the first URL in the email

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Open and preprocess an email
email_filename = "email_name.eml" # ENTER THE EMAIL FILE NAME HERE
with open(email_filename, "rb") as email_byes:
    mail = email_byes.read()
    mail = preprocessor.preprocess_email(mail)
    # Print the extracted subject, header, body, and URLs
    """print("Subject:", mail["subject"])
    print("Headers:")
    print(mail["headers"])
    print("Body:")
    print(mail["body"])
    print("URLS:")
    print(mail["urls"])"""
    # Gather additional information about URLs in the email
if len(mail["urls"]) > 0:
    # Call remote API to gather online URL information
    url_to_analyze = mail["urls"][0]  # for now, we take the first URL
    url_info = url_enricher.get_url_info(url_to_analyze)
else:
    url_info = None

# Classify the email with GPT-4o (LLM-Prompter)

Once we have our preprocessed email and URL information, we call GPT-4o for the classification.

We already did manual prompt engineering and came up with the following prompt:


```
You are a cybersecurity and human-computer interaction expert that has the goal to detect
if an email is legitimate or phishing and help the user understand why a specific email is dangerous (or genuine), in order
to make more informed decisions.
The user will submit the email (headers + subject + body) optionally accompanied by information of the URLs in the email as:
    - server location;
    - VirusTotal scans reporting the number of scanners that detected the URL as harmless.

Your goal is to output a JSON object containing:
- The classification result (label).
- The probability in percentage of the email being phishing (0%=email is surely legitimate, 100%=email is surely phishing) (phishing_probability).
- A list of persuasion principles that were applied by the alliged attacker (if any); each persuasion principle should be an object containing:
    the persuasion principle name (authority, scarcity, etc.),
    the part of the email that makes you say that persuasion principle is being applied;
    a brief rationale for each principle.
- A list of 3 to 5 features that could indicate the danger (or legitimacy) of the email; the explanations must be understandable by users with no cybersecurity or computers expertise.


Desired format:
label: <phishing/legit>
phishing_probability: <0-100%>
persuasion_principles: [array of persuation principles, each having: {{name, specific sentences, rationale}} ]
explanation: [array of 3-5 features explained]

```

Let's modify it a bit and set the code for the API call using Chat Completions ([source](https://platform.openai.com/docs/guides/gpt/chat-completions-api)).

## Generating a warning message with explanation

Now we have a classification for an email that also carries a lot of
information regarding the rationale for the classification and the persuasion principles that might have been used.

Nonetheless, we want to have an explanation message that would be easy to understand also by lay users. Therefore, we create another prompt to further refine this longer explanation in an effective warning message.

We created this prompt:



```
Now take the most relevant feature among the ones in your explanations and construct a brief explanation message (max 50 words) directed to naive users (with no knowledge of cybersecurity) that will follow this structure:`
    1. description of the most relevant phishing feature
    2. explanation of the hazard
    3. consequences of a successful phishing attack
For example, a message that explains that a URL in the email (PHISHING_URL) is imitating another legitimate one (SAFE_URL), would be:
"The target URL [PHISHING_URL] is an imitation of the original one, [SAFE_URL]. This site might be intended to take you to a different place. You might be disclosing private information.”.
Another example of explanation about the domain of a website being suspiciously young would be:
"This website is very young (created [N] days ago). Fraudulent websites have a similar age. There is a potential risk of being cheated if you proceed."
Another example of explaining that the email is suspicious because a domain linked in the email is hosted in a country with bad reputation would be:
"The host of the target website is in [COUNTRY], which is where most attacks originate. Sharing your private information here is risky."

Desired format:
[description of the feature]. [hazard explanation]. [consequences of a successful attack].
```


In [ ]:
import llm_prompter

MODEL = "gpt-4o"

Let's generate an email classification + explanation by using GPT-4o. By changing the parameter _model_ of the function, we can use another GPT model, e.g., GPT-3.5 turbo (see [OpenAI docs](url[https://platform.openai.com/docs/models])).

In [ ]:
# Call GPT-4o for email phishing classification (automatic feature detection)
classification_response, warning_msg = llm_prompter.classify_email(mail, url_info, model=MODEL)

print (classification_response)
print (warning_msg)

## Generate explanations for specific email by priming on the feature to explain

In [ ]:
# Young domain
# Open and preprocess the email
email_filename = "phishing_young_domain.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocessor.preprocess_email(mail)

# We do not use URL information

# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : 'Young domain',
    "description" : 'it contains an URL that leads to a domain that is very new'
}
classification_response, warning_msg = llm_prompter.classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

{'label': 'phishing', 'phishing_probability': '95%', 'persuasion_principles': [{'name': 'Authority', 'specific_sentences': 'BRT Pacchetto in attesa', 'rationale': 'The email impersonates an authoritative courier service to create a sense of legitimacy and urgency.'}, {'name': 'Scarcity', 'specific_sentences': 'Hai (1) pacco in attesa di consegna.', 'rationale': 'The message suggests that there is a limited time to act on the delivery, which can pressure the recipient into taking hasty actions without proper verification.'}, {'name': 'Consistency', 'specific_sentences': 'Pianifica la tua consegna e iscriviti ai nostri avvisi di calendario per evitare che ciò accada di nuovo!!', 'rationale': 'The email prompts the user to take immediate action to resolve an issue, playing on the desire to maintain consistency and follow through with commitments, such as receiving a package.'}], 'explanation': ["The sender's email address ('jessicahwhite47879@gmail.com') does not match the official email 

In [ ]:
# IP Address
# Open and preprocess the email
email_filename = "phishing_IP.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocessor.preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : "URL is IP address",
    "description" : "it contains an URL that is an IP address"
}
classification_response, warning_msg = llm_prompter.classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

In [ ]:
# TLD mispositioned
# Open and preprocess the email
email_filename = "phishing_TLD_mispositioned.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocessor.preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : "Top-Level Domain mispositioned",
    "description" : "it contains an URL with a top-level domain (.com) found as a subdomain"
}
classification_response, warning_msg = llm_prompter.classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)

In [ ]:
# Link mismatch
# Open and preprocess the email
email_filename = "phishing_link_mismatch.eml"
with open(email_filename, "rb") as email_byes:
  mail = email_byes.read()
  mail = preprocessor.preprocess_email(mail)
# Call GPT for email phishing classification (priming the model about the feature to explain)
feature_to_explain = {
    "name" : "Link mismatch",
    "description" : "it contains a displayed link that is different from the actual pointed URL"
}
classification_response, warning_msg = llm_prompter.classify_email(mail, feature_to_explain=feature_to_explain, url_info=None, model=MODEL)

print (classification_response)
print (warning_msg)